In [27]:
#!/bin/bash
!curl -L -o cats-and-dogs-mini-dataset.zip\
https://www.kaggle.com/api/v1/datasets/download/aleemaparakatta/cats-and-dogs-mini-dataset

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

  0 21.8M    0 14949    0     0   8700      0  0:43:56  0:00:01  0:43:55  8700
  0 21.8M    0  149k    0     0  75348      0  0:05:04  0:00:02  0:05:02  428k
  9 21.8M    9 2160k    0     0   716k      0  0:00:31  0:00:03  0:00:28 1655k
 25 21.8M   25 5613k    0     0  1398k      0  0:00:16  0:00:04  0:00:12 2437k
 41 21.8M   41 9373k    0     0  1869k      0  0:00:11  0:00:05  0:00:06 2839k
 58 21.8M   58 12.7M    0     0  2176k      0  0:00:10  0:00:06  0:00:04 3042k
 74 21.8M   74 16.2M    0     0  2378k      0  0:00:09  0:00:07  0:00:02 3319k
 91 21.8M   91 19.9M    0     0  2551k      0  0:0

In [28]:
!mkdir data
import zipfile
with zipfile.ZipFile("cats-and-dogs-mini-dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("dataset") 

In [30]:
import os

# Create the 'data' directory if it doesn't exist
data_dir = 'data'
os.makedirs(data_dir, exist_ok=True)

# Create the 'train' and 'test' subdirectories within 'data'
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

In [31]:
import os

# Define the source and destination paths
source_cat = 'dataset/cats_set'
dest_cat = 'dataset/cat'
source_dog = 'dataset/dogs_set'
dest_dog = 'dataset/dog'

# Rename the directories if they exist
if os.path.exists(source_cat):
  os.rename(source_cat, dest_cat)

if os.path.exists(source_dog):
  os.rename(source_dog, dest_dog)


In [32]:
import os
import random
import shutil

def train_test_split_folder(source_folder, train_folder, test_folder, split_ratio=0.8):
    """
    Splits a folder of images into training and testing sets.

    Args:
        source_folder: Path to the source folder containing subfolders for each class.
        train_folder: Path to the folder where the training set will be saved.
        test_folder: Path to the folder where the testing set will be saved.
        split_ratio: The ratio of images to include in the training set (default is 0.8).
    """

    if not os.path.exists(train_folder):
        os.makedirs(train_folder)
    if not os.path.exists(test_folder):
        os.makedirs(test_folder)

    for class_name in os.listdir(source_folder):
        class_source_path = os.path.join(source_folder, class_name)

        if os.path.isdir(class_source_path):  # Check if it is a directory
            train_class_path = os.path.join(train_folder, class_name)
            test_class_path = os.path.join(test_folder, class_name)

            if not os.path.exists(train_class_path):
                os.makedirs(train_class_path)
            if not os.path.exists(test_class_path):
                os.makedirs(test_class_path)

            images = [f for f in os.listdir(class_source_path) if os.path.isfile(os.path.join(class_source_path, f))]
            random.shuffle(images)
            split_index = int(len(images) * split_ratio)
            train_images = images[:split_index]
            test_images = images[split_index:]

            for image in train_images:
                source_path = os.path.join(class_source_path, image)
                destination_path = os.path.join(train_class_path, image)
                shutil.copy(source_path, destination_path)

            for image in test_images:
                source_path = os.path.join(class_source_path, image)
                destination_path = os.path.join(test_class_path, image)
                shutil.copy(source_path, destination_path)


# Example usage (assuming you have your data organized in a 'data/train' folder):
train_test_split_folder("dataset", "data/train", "data/test")

In [33]:
import os

def count_images_per_folder(root_folder):
  """
  Counts the number of images in each subfolder of a given root folder.

  Args:
    root_folder: The path to the root folder.

  Returns:
    A dictionary where keys are folder paths and values are the number of images in each folder.
  """

  image_counts = {}
  for dirpath, dirnames, filenames in os.walk(root_folder):
    image_count = 0
    for filename in filenames:
      if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
        image_count += 1
    if image_count > 0 :
      image_counts[dirpath] = image_count
  return image_counts

# Example usage
image_counts = count_images_per_folder("data")
for folder, count in image_counts.items():
    print(f"Folder: {folder}, Number of images: {count}")


Folder: data\test\cat, Number of images: 100
Folder: data\test\dog, Number of images: 100
Folder: data\train\cat, Number of images: 400
Folder: data\train\dog, Number of images: 400


In [34]:
!tree data

Folder PATH listing for volume Windows 
Volume serial number is 84AF-5727
C:\USERS\BIT\ONEDRIVE\DESKTOP\DOG-CAT CLASSIFIER\DATA
+---test
�   +---cat
�   +---dog
+---train
    +---cat
    +---dog


In [35]:
import os
import scipy
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' 

import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

In [36]:
print(scipy.__version__)

1.10.1


In [37]:
# Set paths to train and test folders
train_dir = "data/train"
test_dir = "data/test"
image_size = (128, 128)
batch_size = 32

In [38]:
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)


train_data = datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

test_data = datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

Found 640 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


In [39]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # binary classification
])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 63, 63, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 30, 30, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_8 (Conv2D)           (None, 28, 28, 128)       73856     
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 14, 14, 128)      

In [41]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Training the Model
checkpoint_path = 'dog_cat_cnn_model.keras.h5'
checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_accuracy', save_best_only=True)

model.fit(
    train_data,
    epochs=10,
    validation_data=test_data,
    callbacks=[checkpoint]
)

Epoch 1/10
20/20 [==============================] - 15s 636ms/step - loss: 0.6930 - accuracy: 0.5172 - val_loss: 0.6938 - val_accuracy: 0.5000
Epoch 2/10


c:\Users\BIT\OneDrive\Desktop\Dog-cat classifier\.venv\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


20/20 [==============================] - 14s 701ms/step - loss: 0.6920 - accuracy: 0.4984 - val_loss: 0.6926 - val_accuracy: 0.5000
Epoch 3/10
20/20 [==============================] - 15s 725ms/step - loss: 0.6874 - accuracy: 0.5656 - val_loss: 0.6907 - val_accuracy: 0.5000
Epoch 4/10
20/20 [==============================] - 14s 687ms/step - loss: 0.6798 - accuracy: 0.5844 - val_loss: 0.6898 - val_accuracy: 0.5000
Epoch 5/10
20/20 [==============================] - 13s 622ms/step - loss: 0.6674 - accuracy: 0.5984 - val_loss: 0.6882 - val_accuracy: 0.5750
Epoch 6/10
20/20 [==============================] - 15s 771ms/step - loss: 0.6664 - accuracy: 0.6062 - val_loss: 0.6907 - val_accuracy: 0.6000
Epoch 7/10
20/20 [==============================] - 15s 733ms/step - loss: 0.6513 - accuracy: 0.6422 - val_loss: 0.6837 - val_accuracy: 0.5500
Epoch 8/10
20/20 [==============================] - 14s 725ms/step - loss: 0.6300 - accuracy: 0.6828 - val_loss: 0.6570 - val_accuracy: 0.6250
Epoch 9/10

In [42]:
# Save final model
model.save('dog_cat_final_model.keras')

In [43]:
from tensorflow.keras.models import load_model

# Load the best saved model
model = load_model('dog_cat_final_model.keras')

MODEL EVALUATION

In [44]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#set parameters
val_dir=test_dir #same directory used for training
image_size=(128,128)
batch_size=32

#recreate the validation generator

val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.25)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)
# Evaluate model accuracy
loss, accuracy = model.evaluate(val_generator)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")

Found 50 images belonging to 2 classes.


2/2 [==============================] - 1s 143ms/step - loss: 0.6258 - accuracy: 0.6200
Validation Accuracy: 62.00%


MODELINFERENCING


In [45]:
from tensorflow.keras.preprocessing import image
import numpy as np

def predict_image(img_path,model):
    img=image.load_img(img_path,target_size=(128,128))
    img_array=image.img_to_array(img)/255.0
    img_array=np.expand_dims(img_array,axis=0)

    prediction=model.predict(img_array)[0][0]

    if prediction >0.5:
        print("predictied:Dog")
    else:
        print("predicted:Cat")


In [46]:
predict_image("data/train/dog/dog.4156.jpg", model)

1/1 [==============================] - 0s 187ms/step
predictied:Dog
